In [7]:
import pandas as pd
import numpy as np
import pandas as pd
import json
import sys
import os
import gc
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import ast
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!pip install opendatasets
import opendatasets as od

od.download("https://www.kaggle.com/datasets/arjunvankani/recommendation-system-challenge-music-suggestion")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: siddharthkadam1012
Your Kaggle Key: ··········


100%|██████████| 709M/709M [00:07<00:00, 94.8MB/s]


In [174]:
data=pd.read_csv("/content/recommendation-system-challenge-music-suggestion/recommendation challenge (Music)/train.csv/train.csv")

In [175]:
data.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target'],
      dtype='object')

In [176]:
data=data[["msno","song_id","target"]]
data.iloc[24]

msno       pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=
song_id    n2mNwyApX16qOdG4ye2ehf5Cp5oevcahFh27Jyq/7JQ=
target                                                0
Name: 24, dtype: object

In [177]:
data.isnull().sum()

msno       0
song_id    0
target     0
dtype: int64

In [178]:
data=data.head(10000)

In [179]:
data["song_id"].unique()

array(['BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=',
       'bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=',
       'JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=', ...,
       'SBDpmvkE5OQHwA75K9liYM0JaLsIPQ822tW/vwR87AQ=',
       'WLWYK2ULyKhj/FAWvhEOVQuDlZmsmTkKscwahrXsoBM=',
       'sn/EDcAb6jBU4XNQmbUFOwmivzLKw9syjTS6/7yLcno='], dtype=object)

In [180]:
user_songs=data.pivot_table(index="msno",columns=["song_id"],values="target")
user_songs

song_id,++QfzyM/LiFaCuvkDFK/wJe13ZEMgTgAaVCcolo7nnY=,++zVwIlQiLwQOLumYfwBvz0z8XQ+/Gu7Wkry54DDrxk=,+/YtSsv1Gg47x3BC6OLRdHzQvi9ZndPmmN0MNAPaqRI=,+00skobQwnJk5A7n7k3JYO7Qcs+wiYDGvbHZ33K8dHk=,+0mTGQqRUruyn6ovws2qJ/PCpz+WlQ6QB8tc9psDuK4=,+11cGGhMIN0QqBhZoO3ijrUr/pMXwM1zEQzYWWE00OA=,+13MfPvGv321Qw5smYDQ4UTxu1OTjQMRiLcbJAtdECo=,+1pgqoHhV/VuQ1e3xLffp8Uhv6F8B83bJcB1qbFYW34=,+1vkFae+/IYydNdy+pvy+Gl+tt9BvoBlRPu/92fAiEQ=,+2qVVryAzuy9kT6o3lXh8AlznDQaGWYWq4LfBj9Hn3w=,...,zvVHxRErM+EiTX/0bxbCS8TrGJOBT1+PJ8XcfsuXhaE=,zvZrBE1h85EUOTllHRHvl3IefY2Pj79jyYqQ0B3JLb8=,zvjlLoB+A+0WiqVyk+My4kEpKyV5d76Dl1GBXtL05IM=,zwHx/yx5/IKT7gXsde2kj4riuLm4Adexkm+7hzL7l4c=,zwIahUaR94kKTjafis71QLKWVTcnNoPNpGuRzKqZF8w=,zwZhnLk6r2uBufSyllEq8O5GnkPbsRYjOZOSS7QtN54=,zx4aCmk+b/uPmvESsG8ekvi7qWmifSPNMXg1i8wyG8Q=,zygwYWPuTDYl0zrQK5zXtnZzISUneA78GaQgj3kSSfs=,zyv9JGeR75KKL6PNICRXNWEsd1RwYN+k2MMbUQyZ2t0=,zz3O0U0lxl24BQBWM+m5LZ6Pur0uKQZsURIKIyoNkgE=
msno,,,,,,,,,,,,,,,,,,,,,
+2XBp2p9atTpyVPWac3SEEW41i8Xrnl7Utl/h8T4iOo=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
+4xaTltGXsNattBO89s5blOroev2V5i1M1kP0ZJpkyA=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
+50b3keUs/Akq4t58o8FpVPYx8TbKDYPfDlCvq3pVXo=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
+6qXndZ2nMn0cyAWZlBfOBGVqolfO2CRo1MdVznK7Kg=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
+9U5XsTQd6B+ol1h5wm87oMz/+cas2Eu3ig6h7HyH8s=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zr2MEjwpv7bxzJ61TLLBPUoyoSnRqrD8p9BTyUIEXBE=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zryLkfSh02jdklYR1Ex0MuzMBxXYbjN2JM1Ca9mmQi0=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ztnbkP0Dpd0D8XA/QRMYRbl4E8prHJkoGbV0YbmD0Mw=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
#filling the NaN values with 0
user_songs=user_songs.fillna(-1)
user_songs

song_id,++QfzyM/LiFaCuvkDFK/wJe13ZEMgTgAaVCcolo7nnY=,++zVwIlQiLwQOLumYfwBvz0z8XQ+/Gu7Wkry54DDrxk=,+/YtSsv1Gg47x3BC6OLRdHzQvi9ZndPmmN0MNAPaqRI=,+00skobQwnJk5A7n7k3JYO7Qcs+wiYDGvbHZ33K8dHk=,+0mTGQqRUruyn6ovws2qJ/PCpz+WlQ6QB8tc9psDuK4=,+11cGGhMIN0QqBhZoO3ijrUr/pMXwM1zEQzYWWE00OA=,+13MfPvGv321Qw5smYDQ4UTxu1OTjQMRiLcbJAtdECo=,+1pgqoHhV/VuQ1e3xLffp8Uhv6F8B83bJcB1qbFYW34=,+1vkFae+/IYydNdy+pvy+Gl+tt9BvoBlRPu/92fAiEQ=,+2qVVryAzuy9kT6o3lXh8AlznDQaGWYWq4LfBj9Hn3w=,...,zvVHxRErM+EiTX/0bxbCS8TrGJOBT1+PJ8XcfsuXhaE=,zvZrBE1h85EUOTllHRHvl3IefY2Pj79jyYqQ0B3JLb8=,zvjlLoB+A+0WiqVyk+My4kEpKyV5d76Dl1GBXtL05IM=,zwHx/yx5/IKT7gXsde2kj4riuLm4Adexkm+7hzL7l4c=,zwIahUaR94kKTjafis71QLKWVTcnNoPNpGuRzKqZF8w=,zwZhnLk6r2uBufSyllEq8O5GnkPbsRYjOZOSS7QtN54=,zx4aCmk+b/uPmvESsG8ekvi7qWmifSPNMXg1i8wyG8Q=,zygwYWPuTDYl0zrQK5zXtnZzISUneA78GaQgj3kSSfs=,zyv9JGeR75KKL6PNICRXNWEsd1RwYN+k2MMbUQyZ2t0=,zz3O0U0lxl24BQBWM+m5LZ6Pur0uKQZsURIKIyoNkgE=
msno,,,,,,,,,,,,,,,,,,,,,
+2XBp2p9atTpyVPWac3SEEW41i8Xrnl7Utl/h8T4iOo=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
+4xaTltGXsNattBO89s5blOroev2V5i1M1kP0ZJpkyA=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
+50b3keUs/Akq4t58o8FpVPYx8TbKDYPfDlCvq3pVXo=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
+6qXndZ2nMn0cyAWZlBfOBGVqolfO2CRo1MdVznK7Kg=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
+9U5XsTQd6B+ol1h5wm87oMz/+cas2Eu3ig6h7HyH8s=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zr2MEjwpv7bxzJ61TLLBPUoyoSnRqrD8p9BTyUIEXBE=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
zryLkfSh02jdklYR1Ex0MuzMBxXYbjN2JM1Ca9mmQi0=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
ztnbkP0Dpd0D8XA/QRMYRbl4E8prHJkoGbV0YbmD0Mw=,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


Songs-Based

cosine Similarity

In [182]:
songs_similarity=cosine_similarity(user_songs.T)
songs_similarity

array([[1.        , 0.99166929, 0.99166929, ..., 0.99166929, 0.99074664,
        0.99074664],
       [0.99166929, 1.        , 0.9993831 , ..., 0.9993831 , 0.99845827,
        0.99845827],
       [0.99166929, 0.9993831 , 1.        , ..., 0.9993831 , 0.99845827,
        0.99845827],
       ...,
       [0.99166929, 0.9993831 , 0.9993831 , ..., 1.        , 0.99845827,
        0.99845827],
       [0.99074664, 0.99845827, 0.99845827, ..., 0.99845827, 1.        ,
        0.99753391],
       [0.99074664, 0.99845827, 0.99845827, ..., 0.99845827, 0.99753391,
        1.        ]])

In [183]:
songs_similarity=pd.DataFrame(songs_similarity,index=user_songs.columns,columns=user_songs.columns)
songs_similarity

song_id,++QfzyM/LiFaCuvkDFK/wJe13ZEMgTgAaVCcolo7nnY=,++zVwIlQiLwQOLumYfwBvz0z8XQ+/Gu7Wkry54DDrxk=,+/YtSsv1Gg47x3BC6OLRdHzQvi9ZndPmmN0MNAPaqRI=,+00skobQwnJk5A7n7k3JYO7Qcs+wiYDGvbHZ33K8dHk=,+0mTGQqRUruyn6ovws2qJ/PCpz+WlQ6QB8tc9psDuK4=,+11cGGhMIN0QqBhZoO3ijrUr/pMXwM1zEQzYWWE00OA=,+13MfPvGv321Qw5smYDQ4UTxu1OTjQMRiLcbJAtdECo=,+1pgqoHhV/VuQ1e3xLffp8Uhv6F8B83bJcB1qbFYW34=,+1vkFae+/IYydNdy+pvy+Gl+tt9BvoBlRPu/92fAiEQ=,+2qVVryAzuy9kT6o3lXh8AlznDQaGWYWq4LfBj9Hn3w=,...,zvVHxRErM+EiTX/0bxbCS8TrGJOBT1+PJ8XcfsuXhaE=,zvZrBE1h85EUOTllHRHvl3IefY2Pj79jyYqQ0B3JLb8=,zvjlLoB+A+0WiqVyk+My4kEpKyV5d76Dl1GBXtL05IM=,zwHx/yx5/IKT7gXsde2kj4riuLm4Adexkm+7hzL7l4c=,zwIahUaR94kKTjafis71QLKWVTcnNoPNpGuRzKqZF8w=,zwZhnLk6r2uBufSyllEq8O5GnkPbsRYjOZOSS7QtN54=,zx4aCmk+b/uPmvESsG8ekvi7qWmifSPNMXg1i8wyG8Q=,zygwYWPuTDYl0zrQK5zXtnZzISUneA78GaQgj3kSSfs=,zyv9JGeR75KKL6PNICRXNWEsd1RwYN+k2MMbUQyZ2t0=,zz3O0U0lxl24BQBWM+m5LZ6Pur0uKQZsURIKIyoNkgE=
song_id,,,,,,,,,,,,,,,,,,,,,
++QfzyM/LiFaCuvkDFK/wJe13ZEMgTgAaVCcolo7nnY=,1.000000,0.991669,0.991669,0.990747,0.990747,0.990435,0.990747,0.991669,0.991669,0.990747,...,0.990747,0.989201,0.990435,0.990747,0.989513,0.990747,0.973473,0.991669,0.990747,0.990747
++zVwIlQiLwQOLumYfwBvz0z8XQ+/Gu7Wkry54DDrxk=,0.991669,1.000000,0.999383,0.998458,0.998458,0.998149,0.998458,0.999383,0.999383,0.998458,...,0.998458,0.996915,0.998149,0.998458,0.997225,0.998458,0.982424,0.999383,0.998458,0.998458
+/YtSsv1Gg47x3BC6OLRdHzQvi9ZndPmmN0MNAPaqRI=,0.991669,0.999383,1.000000,0.998458,0.998458,0.998149,0.998458,0.999383,0.999383,0.998458,...,0.998458,0.996915,0.998149,0.998458,0.997225,0.998458,0.981190,0.999383,0.998458,0.998458
+00skobQwnJk5A7n7k3JYO7Qcs+wiYDGvbHZ33K8dHk=,0.990747,0.998458,0.998458,1.000000,0.997534,0.997225,0.997534,0.998458,0.998458,0.997534,...,0.997534,0.995991,0.997225,0.997534,0.996301,0.997534,0.980271,0.998458,0.997534,0.997534
+0mTGQqRUruyn6ovws2qJ/PCpz+WlQ6QB8tc9psDuK4=,0.990747,0.998458,0.998458,0.997534,1.000000,0.997225,0.997534,0.998458,0.998458,0.997534,...,0.997534,0.995991,0.997225,0.997534,0.996301,0.997534,0.980271,0.998458,0.997534,0.997534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zwZhnLk6r2uBufSyllEq8O5GnkPbsRYjOZOSS7QtN54=,0.990747,0.998458,0.998458,0.997534,0.997534,0.997225,0.997534,0.998458,0.998458,0.997534,...,0.997534,0.995991,0.997225,0.997534,0.996301,1.000000,0.980271,0.998458,0.997534,0.997534
zx4aCmk+b/uPmvESsG8ekvi7qWmifSPNMXg1i8wyG8Q=,0.973473,0.982424,0.981190,0.980271,0.980271,0.979957,0.980271,0.981190,0.981190,0.980271,...,0.980271,0.978723,0.979957,0.980271,0.979038,0.980271,1.000000,0.981190,0.980271,0.980271
zygwYWPuTDYl0zrQK5zXtnZzISUneA78GaQgj3kSSfs=,0.991669,0.999383,0.999383,0.998458,0.998458,0.998149,0.998458,0.999383,0.999383,0.998458,...,0.998458,0.996915,0.998149,0.998458,0.997225,0.998458,0.981190,1.000000,0.998458,0.998458


In [184]:
def predict_rating_item_item(data,user,song_id,item_item_similarity):
  user_data=data[data["msno"]==user]
  user_data.index=user_data["song_id"]


  df=pd.DataFrame(item_item_similarity.loc[song_id]).sort_values(by=song_id,ascending=False)
  df=df.loc[user_data["song_id"]].sort_values(by=song_id,ascending=False).head(100)
  df["target"]=user_data["target"]
  pred= ((df["target"]*df[song_id]).sum())/(df[song_id].sum())
  return pred

In [185]:
user="pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM="
predicted_rating=[]
for song_id in data["song_id"].unique():
  pred=predict_rating_item_item(data,user,song_id,songs_similarity)
  pred_dict={"song_id":song_id,"predicted":pred}
  predicted_rating.append(pred_dict)



<ipython-input-184-2b3c3f5e7e4e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"]=user_data["target"]


In [186]:
predicted_rating=pd.DataFrame(predicted_rating)


In [187]:
predicted_rating

,song_id,predicted
0,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,0.333128
1,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,0.333128
2,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,0.333128
3,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,0.333128
4,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,0.333128
...,...,...
5767,c1XhogBZkWcjQv8NfrQpFNkGScf2X346zCmwhScBSUs=,0.333128
5768,eExyzh5FVJYlRFGr4edl1oaGcUYgpfcYA8KKyS6nypU=,0.333128
5769,SBDpmvkE5OQHwA75K9liYM0JaLsIPQ822tW/vwR87AQ=,0.333128
5770,WLWYK2ULyKhj/FAWvhEOVQuDlZmsmTkKscwahrXsoBM=,0.333128


In [189]:
predicted_rating["predicted"]

0       0.333128
1       0.333128
2       0.333128
3       0.333128
4       0.333128
          ...   
5767    0.333128
5768    0.333128
5769    0.333128
5770    0.333128
5771    0.333128
Name: predicted, Length: 5772, dtype: float64